In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../Licencjat/licencjat.csv", delimiter=";")  

C:\Users\Ester\AppData\Local\Temp\ipykernel_388\452193696.py:1: DtypeWarning: Columns (21,95,107) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../Licencjat/licencjat.csv", delimiter=";")


In [3]:
data = data[[
    "country", "age", "isced1997_r", "mar_stat", "hhsize", "partnerinhh", "ch001_", "sphus",
    "chronic_mod", "eurod",
    "co007_", "thinc_m", "income_pct_w6", "wave"
]]

In [4]:
data.head()

,country,age,isced1997_r,mar_stat,hhsize,partnerinhh,ch001_,sphus,chronic_mod,eurod,co007_,thinc_m,income_pct_w8,wave
0,11,"51,9000015258789",3,1,2,1,4,1,0,0,2,"28721,849609375",-13,1
1,11,"54,7000007629395",3,1,2,1,4,5,1,9,3,"26432,822265625",-13,2
2,11,"48,9000015258789",3,1,2,1,4,1,0,1,2,"28721,849609375",-13,1
3,11,"51,7000007629395",3,1,2,1,4,2,1,3,3,"26432,822265625",-13,2
4,11,"59,7000007629395",5,6,1,3,2,3,1,1,4,"9167,8212890625",-13,4


### 1. Czyszczenie danych

In [6]:
data = data[data['wave'] == 8]

In [7]:
data = data[data['country'].isin([15, 29])]

In [8]:
data["thinc_m"] = data["thinc_m"].str.replace(",", ".")
data["age"] = data["age"].str.replace(",", ".")
data["age"] = data["age"].astype(float)
data["thinc_m"] = data["thinc_m"].astype(float)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4985 entries, 207816 to 425272
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        4985 non-null   int64  
 1   age            4985 non-null   float64
 2   isced1997_r    4985 non-null   int64  
 3   mar_stat       4985 non-null   int64  
 4   hhsize         4985 non-null   int64  
 5   partnerinhh    4985 non-null   int64  
 6   ch001_         4985 non-null   int64  
 7   sphus          4985 non-null   int64  
 8   chronic_mod    4985 non-null   int64  
 9   eurod          4985 non-null   int64  
 10  co007_         4985 non-null   int64  
 11  thinc_m        4985 non-null   float64
 12  income_pct_w8  4985 non-null   int64  
 13  wave           4985 non-null   int64  
dtypes: float64(2), int64(12)
memory usage: 584.2 KB


In [10]:
data = data.drop(columns=['country'])

In [11]:
data[data < 0] = np.nan

In [12]:
data = data.dropna()

In [14]:
data.shape

(4539, 13)

### 2. Feature Engineering

In [16]:
data.columns

Index(['age', 'isced1997_r', 'mar_stat', 'hhsize', 'partnerinhh', 'ch001_',
       'sphus', 'chronic_mod', 'eurod', 'co007_', 'thinc_m', 'income_pct_w8',
       'wave'],
      dtype='object')

### 3. Modelowanie

In [19]:
X = data[['age', 'isced1997_r', 'mar_stat', 'hhsize', 'partnerinhh', 'ch001_',
       'sphus', 'chronic_mod', 'eurod', 'thinc_m', 'income_pct_w8',
       'wave']]
y = data['co007_']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train,
        eval_set = [(X_train, y_train), (X_test, y_test)],
        verbose = True)
xgb_pred = xgb.predict(X_test)

[0]	validation_0-rmse:0.83031	validation_1-rmse:0.84851
[1]	validation_0-rmse:0.77954	validation_1-rmse:0.81050
[2]	validation_0-rmse:0.74665	validation_1-rmse:0.78881
[3]	validation_0-rmse:0.72330	validation_1-rmse:0.77507
[4]	validation_0-rmse:0.70783	validation_1-rmse:0.76980
[5]	validation_0-rmse:0.69295	validation_1-rmse:0.76431
[6]	validation_0-rmse:0.68453	validation_1-rmse:0.76020
[7]	validation_0-rmse:0.67908	validation_1-rmse:0.76062
[8]	validation_0-rmse:0.66760	validation_1-rmse:0.75883
[9]	validation_0-rmse:0.66278	validation_1-rmse:0.75879
[10]	validation_0-rmse:0.65449	validation_1-rmse:0.75877
[11]	validation_0-rmse:0.65283	validation_1-rmse:0.75848
[12]	validation_0-rmse:0.64518	validation_1-rmse:0.75750
[13]	validation_0-rmse:0.64273	validation_1-rmse:0.75684
[14]	validation_0-rmse:0.63799	validation_1-rmse:0.75834
[15]	validation_0-rmse:0.63683	validation_1-rmse:0.75967
[16]	validation_0-rmse:0.62799	validation_1-rmse:0.75895
[17]	validation_0-rmse:0.62254	validation

In [25]:
np.sqrt(metrics.r2_score(y_test,xgb_pred))

0.5473241863400556